# PROJECT
## Applications of Big Data

### 0. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

### 3. Data Modeling

#### 3.1. Dự đoán xu hướng tuyển dụng

In [2]:
df = pd.read_csv('dataset/data_clean.csv')
df.head()

,Job Id,Qualifications,location,Country,Work Type,Company Size,Job Posting Date,Preference,Job Title,Role,...,Job Description,Benefits,skills,Responsibilities,Company,Salary_avg,Experience_avg,Day,Month,Year
0,1089843540111562,M.Tech,Douglas,Isle of Man,Intern,26801,2022-04-24,Female,Digital Marketing Specialist,Social Media Manager,...,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,79.0,10.0,24,4,2022
1,398454096642776,BCA,Ashgabat,Turkmenistan,Intern,100340,2022-12-19,Female,Web Developer,Frontend Web Developer,...,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,86.0,7.0,19,12,2022
2,481640072963533,PhD,Macao,"Macao SAR, China",Temporary,84525,2022-09-14,Male,Operations Manager,Quality Control Manager,...,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,82.5,6.0,14,9,2022
3,688192671473044,PhD,Porto-Novo,Benin,Full-Time,129896,2023-02-25,Female,Network Engineer,Wireless Network Engineer,...,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,78.0,7.5,25,2,2023
4,117057806156508,MBA,Santiago,Chile,Intern,53944,2022-10-11,Female,Event Manager,Conference Manager,...,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,75.5,6.5,11,10,2022


##### 3.1.1. Data Preparing

In [3]:
df['Job Posting Date'] = pd.to_datetime(df['Job Posting Date'])

In [4]:
df['Job Posting Date'].describe()

C:\Users\HoaTX\AppData\Local\Temp\ipykernel_2992\2007897400.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['Job Posting Date'].describe()


count                 1615940
unique                    731
top       2021-11-14 00:00:00
freq                     2378
first     2021-09-15 00:00:00
last      2023-09-15 00:00:00
Name: Job Posting Date, dtype: object

Thêm cột `Target1` để dự đoán xu hướng trong thị trường tuyển dụng dựa trên dữ liệu.

Ngày đăng tuyển dụng cuối cùng được thu thập là `15/09/2023` ta sẽ chọn giá trị cho cột `Target1` là `1` với những công việc tuyển dụng từ ngày `02/08/2023` và ngược lại là `0`. Nghĩa là trong `45` ngày gần nhất đăng tuyển để dự đoán xu hướng tuyển dụng.

In [5]:
df['Target1'] = df['Job Posting Date'].apply(lambda x: 1 if x >= pd.to_datetime('2023-08-02') else 0)

In [6]:
features = ['Work Type', 'Company Size', 'Job Title', 'Experience_avg']
df_model = df[features + ['Target1']]

In [7]:
df_model = pd.get_dummies(df_model, columns=['Job Title', 'Work Type'], prefix=['Job_Title', 'Work_Type'], drop_first=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('Target1', axis=1), df_model['Target1'], test_size=0.2, random_state=42)

##### 3.1.2. Naive Bayes

Mô hình `MultinomialNB` là một mô hình học máy có giám sát sử dụng cho bài toán phân loại văn bản. Nó dựa trên nguyên tắc của `Naive Bayes` và được sử dụng cho các tập dữ liệu có các đặc trưng rời rạc, chẳng hạn như đếm số lần xuất hiện của các từ trong văn bản. Mô hình xây dựng bộ "từ vựng" từ dữ liệu huấn luyện và tính xác suất xuất hiện của các từ trong "từ vựng" trong các lớp khác nhau. Sau đó, sử dụng các xác suất này để huấn luyện mô hình và phân loại các văn bản mới bằng cách dự đoán lớp có xác suất cao nhất cho mỗi văn bản. 

In [9]:
model1 = MultinomialNB()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

acc_1 = accuracy_score(y_test, y_pred1)
prec_1 = precision_score(y_test, y_pred1)
rec_1 = recall_score(y_test, y_pred1)
f1_1 = f1_score(y_test, y_pred1)

print('accuracy_score: ', acc_1)
print('precision_score: ', prec_1)
print('recall_score: ', rec_1)
print('f1_score: ', f1_1)

e:\Conda\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score:  0.9381103258784361
precision_score:  0.0
recall_score:  0.0
f1_score:  0.0
